# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

Do not modify or remove any of the code in these cells.

In [1]:
!pip install imblearn

In [2]:
!pip install nltk

In [3]:
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple


In [4]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv

from string import punctuation
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


stopwords = nltk.corpus.stopwords.words('english')

from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\27714\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\27714\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') # no labels

train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [6]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


## Overview

Refer to this diagram to guide you while you build your model. Some of the steps will be fleshed out in this template.

![Overview](process_overview_final.jpg)

## Basic preprocessing

Here is a template you may use for your initial base model. 

### Removing URL's
Write a function that removes URL's from a single tweet. 

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet with URL's removed as a single object

In [7]:
# define function that removes URL from single tweet

import re
def remove_urls(text):
  """Remove URLs from a text string"""
  return re.sub(r"http\S+", "", text)

**Original object:**

In [8]:
tweet = train['message'][2]
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [9]:
remove_urls(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '

**Expected output:**
```python
remove_urls(tweet)

>>'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '
```

### Convert to lowercase

Write a function that converts a single tweet to lowercase.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in lowercase as a single object

In [10]:
def to_lower(text):
    text = text.lower()
    return text 

**Original object:**

In [11]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [12]:
to_lower(tweet)

'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**

```python 
to_lower(tweet)
>> 'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

### Stemming or lemmatising

Write a function that derives the root words for each of the words in a tweet. You may use stemming _or_ lemmatising here.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in root words as a _single_ object

In [13]:
def get_roots(text):
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(token) for token in text.split(' ')]
    text = [lemmatizer.lemmatize(token, 'v') for token in text]
    text = ' '.join(text)
   
    return text

**Original object:**

In [14]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

### Other preprocessing techniques 

Come back here once you have submitted your base model to add any other preprocessing functions yuo wish to add

In [15]:
# YOUR CODE HERE






**Expected output:**
```python
get_roots(tweet)
```
Using `SnowballStemmer()`
```python
>> 'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

Using `WordNetLemmatizer()`
```python
>> 'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'
```

### Tokenising 

Write a function that tokenises a single tweet into a list of tokens.

**Function input:**
- A single string object (tweet) 

**Function output:**
- A list of tokens (objects).

In [16]:


def get_tokens(text):
    
    text = word_tokenize(text)  
    return text

**Original object:**

In [17]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [18]:
get_tokens(tweet)

['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it',
 '’',
 's',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']

**Expected output:**
```python
get_tokens(tweet)

>> ['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']
```

Besides the preprocessing methods we've already discussed, can you think of how our data can be improved at this stage?

### Vectorising 

Use the code from the webinar to define your vecotiser. For more details on this function have a look at the `sklearn`
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">package documentation on the function</a>. Recall that this function, unlike the others before takes an _**entire column**_ as an argument.

### Putting all the functions together 

Test your cleaning functions on a single tweet. Perform the cleaning procedure on this tweet from start to finish. Make sure the output looks as you would expect. Store the cleaned tweet in `tweet_final`. Don't worry about vectorising for now.

**_Include any other cleaning procedures you add after your base model_**

**Original object:**

In [19]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [20]:
# clean tweet from start to finish
 ###tweet1 = remove_urls(tweet)

    
def remove_urls(tweet):
 
  return re.sub(r"http\S+", "", tweet)
    
         
    
  
  ##tweet2 = #... YOUR CODE HERE
#.
#.
#.

# cleaned tweet
####tweet_final

###def tweet_final(tweet):






**Expected output:**
```python
tweet_final
>>['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']
```

### Preprocessing and cleaning entire column of tweets

Now that we know how to clean a single tweet, we can use `apply()` to clean the entire column in the dataframe. This is what the column looks like to begin with.

In [21]:
train['message'].head()

0    PolySciMajor EPA chief doesn't think carbon di...
1    It's not like we lack evidence of anthropogeni...
2    RT @RawStory: Researchers say we have three ye...
3    #TodayinMaker# WIRED : 2016 was a pivotal year...
4    RT @SoyNovioDeTodas: It's 2016, and a racist, ...
Name: message, dtype: object

Write code that applies your cleaning functions to the entire `message` column of the dataframe using `.apply()`. Do not vectorise just yet. Store this in a new column, `train['message_clean']`. 

In [22]:
# applying cleaning functions to entire column of dataframe (NO VECTORISING)

# YOUR CODE HERE - use apply with all your cleaning functions 


def cleaning_tweet(text):

    text = re.sub('https?:\/\/\S+', '', text)                 # Removing url
    ####text = re.sub('@[A-Za-z0–9]+', '', text)              #Removing @mentions
    text = re.sub('[^a-zA-Z]',' ', text)   
    ##text = re.sub('#', '', text)                              # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text)                        # Removing RT
    text = str(text).lower()                                  # to lower case
    text = word_tokenize(text)                                # Tokenisation
    
    text = [item for item in text if item not in stopwords]  #Removing all stopwords
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word=w,pos='v') for w in text]
    text = [i for i in text if len(i)>2] 
    text = ' '.join(text)  
    ####text = ''.join(i for i in text if i not in punctuation)   # Remove Puncuation
    text = re.sub('[^a-z]', ' ',text)                         # Removing non-alphabets

    return text

In [23]:
#Apply data cleaning function to data
train['clean_message'] = train['message'].map(cleaning_tweet)


In [24]:
train['clean_message'].head()

0    polyscimajor epa chief think carbon dioxide ma...
1         like lack evidence anthropogenic global warm
2    rawstory researchers say three years act clima...
3    todayinmaker wire pivotal year war climate change
4    soynoviodetodas racist sexist climate change d...
Name: clean_message, dtype: object

**Expected output:**

```python
train['message_clean'].head()

>>  0    [polyscimajor, epa, chief, does, n't, think, c...
    1    [it, not, like, we, lack, evid, of, anthropoge...
    2    [rt, @, rawstory, :, research, say, we, have, ...
    3    [#, todayinmaker, #, wire, :, 2016, was, a, pi...
    4    [rt, @, soynoviodetodas, :, it, 2016, ,, and, ...
Name: message_clean, dtype: object
```

### Vectorising 

Vectorise your cleaned data using the vectoriser you defined earlier. Don't forget to fit the vectoriser to the data you just cleaned. Store your vectorised data in `train_vec`.

In [25]:
y = train['sentiment']
#X = train['clean_Message']
X= train['message']

In [26]:
# vectorise your cleaned data

# YOUR CODE HERE 
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

## Fitting a simple model on training data and evaluate its performance

Fit a model on your cleaned data. Use `train_vec` as your features. Note that you need to convert `train_vec` to an array. 

You are familiar with this process, so there is less guidance here. You may also refer to train notebooks and the webinar pdf as guidance.

Your basic model should be a logistic regression, however if you try different models you will also perform the following procedure.

1. Split the training data into features and labels.
2. Split the training data further into training and validation data.
3. Fit the model on the training subset. 
4. Predict on the validation subset.
5. Calculate the performance metrics on the validation predictions.
6. Select a model based on validation performance (when you have more than one model).
8. Clean the test data.
9. Predict on the cleaned test data.
7. Write a csv that matches `sample_submission.csv`.
8. Submit to Kaggle.

In [27]:
# YOUR CODE HERE 

#X_train,X_test,y_train,y_test = train_test_split(X_vectorized,y,test_size=.30,shuffle=True, stratify=y, random_state=44)
from sklearn.model_selection import RepeatedKFold
kf = RepeatedKFold(n_splits=10, n_repeats=20, random_state=None) 

for train_index, test_index in kf.split(X_vectorized):
    #print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X_vectorized[train_index], X_vectorized[test_index] 
    y_train, y_test = y[train_index], y[test_index]



## Suggestions on how to improve the model - Extras 

Here are some ideas on how you can improve your model. Try some of them out once you have submitted your base model to Kaggle. Only use the methods you find improve your model performance.

 - Additional preprocessing 
     - Use some of the methods in the notebooks to balance the data
     - Replace links with the text `Link` to show the machine that there is a link present without the extraneous detail of the link text
     - Remove punctuation
     - Correct spelling
     - Bag of words vs. N-grams - which is better?
     - Try `TweetTokenizer()` - how does this differ from the tokenisers you are familiar with?
 - Consider which other models could do a better job (refer to course resources)
 - Consider some further feature engineering:
     - What other attributes of the tweets may be useful?
     - Should we perform feature selection given their size and sparseness of the vectorised tweets?
 - If you've mastered the above, try implementing hyperparameter tuning methods.
 - Compare all the models you try and remember to submit **the best one** based on test performance (f1-score using macro averaging). 
 
You may use the space below to try out these "Extras".

In [28]:
# Extras
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_smote,y_smote = oversample.fit_resample(X_train,y_train)
####smote =SMOTE()
###X_smote,y_smote =smote.fit_sample(X_train,y_train)




In [29]:
from collections import Counter
print ("Before smote",Counter(y_train))
print ("after smote",Counter(y_smote))

Before smote Counter({1: 7671, 2: 3277, 0: 2127, -1: 1163})
after smote Counter({1: 7671, 2: 7671, 0: 7671, -1: 7671})


## How/why is the model bad?

Do you have any hypotheses as to how you can improve the model further? Use this space to investigate with mindful EDA if necessary.

In [30]:
# EDA
# YOUR CODE HERE 

rfc = RandomForestClassifier(n_estimators=200, random_state=44)
rfc.fit(X_smote, y_smote)
rfc_pred = rfc.predict(X_test)



In [31]:
f1_score(y_test, rfc_pred,average="macro")

0.6251068376366895

Be sure to check whether each additional technique you apply actually improves your model performance.

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#print(confusion_matrix(y_val, rfc_pred))
print(classification_report(y_test, rfc_pred))
#print(accuracy_score(y_val, rfc_pred))

              precision    recall  f1-score   support

          -1       0.93      0.29      0.45       133
           0       0.54      0.42      0.47       226
           1       0.75      0.85      0.80       859
           2       0.76      0.81      0.78       363

    accuracy                           0.73      1581
   macro avg       0.74      0.59      0.63      1581
weighted avg       0.74      0.73      0.72      1581



In [33]:
testx = test['message']
test_vect = vectorizer.transform(testx)

In [34]:
y_pred = rfc.predict(test_vect)

In [35]:
test['sentiment'] = y_pred
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


## Submit your best model predictions 

Don't forget to submit your the predictions on the best model to Kaggle.

In [36]:
# write csv for best predictions

test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)
